## Semantic Function Calling

In [1]:
#r "nuget:Microsoft.SemanticKernel"
#r "nuget: dotenv.net"
#r "nuget:Microsoft.SemanticKernel.Connectors.OpenAI"
#r "nuget:Microsoft.SemanticKernel.Connectors.AzureOpenAI"

Installed Packages dotenv.net, 3.2.0 Microsoft.SemanticKernel, 1.19.0 Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.19.0 microsoft.semantickernel.connectors.openai, 1.19.0

In [22]:
using dotenv.net;

DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT");
var endpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at {endpoint} with key {apiKey.Substring(0, 5)}...");
     

Using deployment: gpt-4o at https://scribeai.openai.azure.com/ with key 9c439...


In [2]:
using System.ComponentModel;
using Microsoft.SemanticKernel;
using System.Text.Json.Serialization;

public class EmailPlugin
{
    [KernelFunction("send_email")]
    [Description("Sends an email to a recipient.")]
    public async Task SendEmailAsync(
        Kernel kernel,
        List<string> recipientEmails,
        string subject,
        string body
    )
    {
        // Add logic to send an email using the recipientEmails, subject, and body
        // For now, we'll just print out a success message to the console
        Console.WriteLine("Email sent!");
        Console.WriteLine($"Recipients: {string.Join(", ", recipientEmails)}");
        Console.WriteLine($"Subject: {subject}");
        Console.WriteLine($"Body: {body}");
        
    }
}

public class EventModel
{
   [JsonPropertyName("id")]
   public int Id { get; set; }

   [JsonPropertyName("name")]
   public string Name { get; set; }
}

public class CalendarPlugin
{
    [KernelFunction("get_calendar_events")]
    [Description("Gets calendar events for a given date.")]
    public async Task<List<EventModel>> GetCalendarAsync(
        Kernel kernel,
        List<string> recipientEmails,
        System.DateTime date
    )
    {
        // Add logic to get calendar events for recipientEmails on the given date
        // For now, we'll just print out a success message to the console
        Console.WriteLine("Calendar Events");
        Console.WriteLine($"Recipients: {string.Join(", ", recipientEmails)}");
        Console.WriteLine($"date: {date}");

        // Call Graph API to get calendar events !!!!!

        return   new()
            {
                new EventModel { Id = 1, Name = "Soccer practice" },
                new EventModel { Id = 2, Name = "Date night"},
                new EventModel { Id = 3, Name = "Dentist appointment" }
            };
    }
}

# ASK Boss for Vacation

In [12]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;



// Create kernel with an email plugin
var builder = Kernel.CreateBuilder();
builder.Plugins.AddFromType<EmailPlugin>();
builder.Plugins.AddFromType<CalendarPlugin>();
Kernel kernel = builder.Build();

// Create chat completion service

AzureOpenAIChatCompletionService chatCompletionService = new (
    deploymentName: deploymentName,
    apiKey: apiKey,
    endpoint: endpoint
 );

// Enable planning
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};
Console.WriteLine($" --------------------------------");
// Create chat history
var history = new ChatHistory();
history.AddUserMessage("Can you help me Elena write an email for my boss at boss@gmail.com subject: oh mine asking for vacation for days of Oct 15th and 16th in professional tone?");

// Get the response from the AI
var result = await chatCompletionService.GetChatMessageContentAsync(
    history,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel
);

Console.WriteLine($" --------------------------------");
Console.WriteLine($" Email : {result}");
Console.WriteLine($" --------------------------------");



 --------------------------------
Email sent!
Recipients: boss@gmail.com
Subject: Vacation Request for October 15th and 16th
Body: Dear [Boss's Name],

I hope this message finds you well. I am writing to formally request vacation days on October 15th and 16th. I have ensured that all my tasks and responsibilities will be handled in my absence.

Your approval for these days off would be greatly appreciated.

Thank you for considering my request. Please let me know if you need any additional information.

Best regards,
Elena
 --------------------------------
 Email : Your email request for vacation days on October 15th and 16th has been sent to your boss at boss@gmail.com.
 --------------------------------


# ASK Calendar events for vacation time

In [4]:
history.AddUserMessage("What are my events for vacation days,list all of them ");

// Get the response from the AI
var vacresult = await chatCompletionService.GetChatMessageContentAsync(
    history,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel
);
Console.WriteLine($" --------------------------------");
Console.WriteLine($" Vacation events : {vacresult}");

 --------------------------------
 Vacation events : Here are your events for the vacation days (October 15th and 16th):

**October 15th:**
1. Soccer practice
2. Date night
3. Dentist appointment

**October 16th:**
1. Soccer practice
2. Date night
3. Dentist appointment
